# Collecting all details Except Claim Value

## Step 1

In [ ]:

def login():
    """ LOGIN INTO THE WEBSITE WITH ALL DETAILS """
    try:
        driver.get("https://court.martinclerk.com")
        #To load Website Completely
        time.sleep(3)
        #To enter the dates from and to
        DateFrom = driver.find_element_by_id('openedFrom')
        DateFrom.click()
        DateFrom.send_keys("03162020")

        DateTo = driver.find_element_by_id('openedTo')
        DateTo.click()
        DateTo.send_keys("03222020")

        driver.find_element_by_xpath("//*[@id='courTypesButton']").click()
        driver.find_element_by_xpath("//*[@id='courTypesMultiSelect']/ul/li[1]/a/label/input").click()
        time.sleep(2) 
        driver.find_element_by_xpath("//*[@id='courTypesMultiSelect']/ul/li[3]/a/label/input").click()

        #To Get the next DropDownList
        time.sleep(10)
        driver.find_element_by_xpath("//*[@id='courTypesButton']").click()
        time.sleep(2)

        driver.find_element_by_xpath("//*[@id='caseTypesButton']").click()
        driver.find_element_by_xpath("//*[@id='caseTypesMultiSelect']/ul/li[1]/a/label/input").click()
        time.sleep(1)
        #To select the Categories
        for i in range(69,78):
            driver.find_element_by_xpath("//*[@id='caseTypesMultiSelect']/ul/li["+str(i)+"]/a/label/input").click()

        Captcha_Entry_Box = driver.find_element_by_xpath("//*[@id='mainTableContent']/tbody/tr/td/table/tbody/tr[2]/td[2]/div/div[3]/form/input[2]")
        Captcha_Entry_Box.click()

        time.sleep(3)

        captcha_additon = int(input("Enter the additon check the headless browser :\t"))
        Captcha_Entry_Box.send_keys(str(captcha_additon))

        #To click on Search Button
        Search_Button = driver.find_element_by_xpath("//*[@id='searchButton']")
        Search_Button.click()

        #To Load all the classes on webpage
        time.sleep(5)
    except Exception as e:
        print("Error at Login() = ",e)
        print("BURN THE CODE AGAIN....!!")

def collect_data_from_search():
    """ COLLECT DATA FROM SEARCH RESULT TO USE FOR ITERATIONS """
    try:
        table = driver.find_element_by_id("gridSearchResults")
        rows = table.find_elements_by_tag_name("tr")

        global data_row
        data_row = []
        for index in range(1,len(rows)):
            data_row.append(rows[index].text)
        print("Lenght of Data Collected From Rows = ",len(data_row))
    except Exception as e:
        print("Error at collect_data_from_search() = ",e)


def Clean_raw_data():
    """ CLEANNING RAW DATA """
    global case_id , bank_name , def_name 
    try:
        #Cleanning the raw_data ...... !!!
        case_id = []
        bank_name = []
        def_name = []
        for i in range(len(raw_data)):
            caseid = raw_data[i].split("-")[0]
            case_id.append(caseid)
            bankname = raw_data[i].split("-")[1].split("vs.")[0]
            bank_name.append(bankname)
            defname = raw_data[i].split("-")[1].split("vs.")[1]
            def_name.append(defname)

        print(len(bank_name),len(case_id),len(def_name))
    except Exception as e:
        print("Error at Clean_raw_data() = ",e)

        
def PDF_download():
    """ DOWNLOADING THE PDF """
    try:
        #Searching For PDF
        table = driver.find_element_by_id("gridDockets")
        rows = table.find_elements_by_tag_name("tr")

        #Gathering Data From GRID_DOCKETS {PDF search}
        gridDockets = []
        for i in range(1,len(rows)):
            gridDockets.append(rows[i].text)
        print("Lenght of Data Collected From Rows = ",len(gridDockets))
        #Searching for PDF present or not
        for index,data in enumerate(gridDockets):
            if ( "VALUE OF REAL PROPERTY OR MORTGAGE FORECLOSURE CLAIM:" in data ):
                print("PDF Found index = ",index)
                #Clicking on the PDF imaged button
                driver.find_element_by_xpath('//*[@id="28896399"]').click()
                time.sleep(3)
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(3)
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(10)
                driver.execute_script('window.print();')
            else:
                pass
    except Exception as e:
        print("Error at PDF_download() = ",e)


"""###############################################"""


from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pdf2image import convert_from_path
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import pyautogui
import requests
import random
import PyPDF2 
import json
import mouse
import time 
import os
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe"
base_dir = os.path.abspath(os.path.dirname("__file__"))

# Different Proxys : =    192.41.71.221:3128   50.246.120.125:8080     
PROXY = "192.41.71.199:3128"

options = webdriver.ChromeOptions()

appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}

options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False 
# options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
path = "C:\\Users\\Dishant\\Desktop\\chromedriver\\chromedriver"
global driver 
driver = webdriver.Chrome(executable_path = path , options = options)

#Step1
login() 

#Step2
collect_data_from_search() 
time.sleep(10)

#Step3
global raw_data , case_date , case_type
try:
    raw_data = []
    case_date = []
    case_type = []
    for i in range(len(data_row)):
        driver.find_element_by_xpath("//*[@id='gridSearchResults']/tbody/tr["+str(i+1)+"]/td[3]/a").click()   
        #Web page load completely
        time.sleep(10)
        data = driver.execute_script("return document.getElementsByClassName('pull-left')[0].innerText")
        time.sleep(3)
        Case_Date = driver.execute_script("return document.getElementsByClassName('clerkfiledate')[1].innerText")
        time.sleep(3)
        Case_Type = driver.execute_script("return document.getElementsByClassName('casetype')[1].innerText")
        time.sleep(3)
        raw_data.append(data)
        case_date.append(Case_Date)
        case_type.append(Case_Type)
        time.sleep(2)
        #to go back to previous page
        driver.execute_script("window.history.go(-1)")
        #to load the page with all classes
        time.sleep(8)
    print(len(raw_data) , len(case_date) , len(case_type))
except Exception as e:
    print("Error at Step 3 = ",e)

#Cleanning Raw Data
Clean_raw_data()

#Cleanning case_date
global new_case_date
new_case_date = []
for i in case_date:
    demo = re.search("\d{1,2}\/\d{1,2}\/\d{1,4}",i).group()
    mm = demo.split("/")[0]
    dd = demo.split("/")[1]
    yy = demo.split("/")[2]
    new_case_date.append(dd+"/"+mm+"/"+yy)
print(len(new_case_date))

driver.quit()

#Creting Dataset
dataset = pd.DataFrame({'CaseId':case_id,'CaseDate':new_case_date,'CaseType':case_type,
                        'BankName':bank_name,'DefendentName':def_name})
#Creating a trial CSV
dataset.to_csv('Martin_County_Try_Data.csv')   

## Step 2

In [ ]:

def login():
    """ LOGIN INTO THE WEBSITE WITH ALL DETAILS """
    try:
        driver.get("https://court.martinclerk.com")
        #To load Website Completely
        time.sleep(3)
        #To enter the dates from and to
        DateFrom = driver.find_element_by_id('openedFrom')
        DateFrom.click()
        DateFrom.send_keys("03162020")

        DateTo = driver.find_element_by_id('openedTo')
        DateTo.click()
        DateTo.send_keys("03222020")

        driver.find_element_by_xpath("//*[@id='courTypesButton']").click()
        driver.find_element_by_xpath("//*[@id='courTypesMultiSelect']/ul/li[1]/a/label/input").click()
        time.sleep(2) 
        driver.find_element_by_xpath("//*[@id='courTypesMultiSelect']/ul/li[3]/a/label/input").click()

        #To Get the next DropDownList
        time.sleep(10)
        driver.find_element_by_xpath("//*[@id='courTypesButton']").click()
        time.sleep(2)

        driver.find_element_by_xpath("//*[@id='caseTypesButton']").click()
        driver.find_element_by_xpath("//*[@id='caseTypesMultiSelect']/ul/li[1]/a/label/input").click()
        time.sleep(1)
        #To select the Categories
        for i in range(69,78):
            driver.find_element_by_xpath("//*[@id='caseTypesMultiSelect']/ul/li["+str(i)+"]/a/label/input").click()

        Captcha_Entry_Box = driver.find_element_by_xpath("//*[@id='mainTableContent']/tbody/tr/td/table/tbody/tr[2]/td[2]/div/div[3]/form/input[2]")
        Captcha_Entry_Box.click()

        time.sleep(3)

        captcha_additon = int(input("Enter the additon check the headless browser :\t"))
        Captcha_Entry_Box.send_keys(str(captcha_additon))

        #To click on Search Button
        Search_Button = driver.find_element_by_xpath("//*[@id='searchButton']")
        Search_Button.click()

        #To Load all the classes on webpage
        time.sleep(5)
    except Exception as e:
        print("Error at Login() = ",e)
        print("BURN THE CODE AGAIN....!!")

def collect_data_from_search():
    """ COLLECT DATA FROM SEARCH RESULT TO USE FOR ITERATIONS """
    try:
        table = driver.find_element_by_id("gridSearchResults")
        rows = table.find_elements_by_tag_name("tr")

        global data_row
        data_row = []
        for index in range(1,len(rows)):
            data_row.append(rows[index].text)
        print("Lenght of Data Collected From Rows = ",len(data_row))
    except Exception as e:
        print("Error at collect_data_from_search() = ",e)
        

def pdf_claim_value():
    """ TO EXTRACT CLAIM VALUE """
    try:

        #Using GLOB module
        fn = glob.glob("C:\\Users\\Dishant\\Downloads\\*.pdf")
        FN = fn[0]
        print("Filename = ",FN)
        filename = FN
        pages = convert_from_path(filename, 500)
        print("Number of Pages found in PDF = ",len(pages))
        matches = ""
        for page in pages:
            page.save('C:\\Users\\Dishant\\Downloads\\property_value.jpg', 'JPEG')
            global property_value
            property_value = str(((pytesseract.image_to_string(Image.open('C:\\Users\\Dishant\\Downloads\\property_value.jpg'))))).lower()
            #print(property_value)
            try:
                matches = re.search("(\\d+\\W)*total estimated value of claim(\\W*\\d*)+", property_value).group()
                matches = matches.split("\n")
                if "total" in matches[0]:
                    matches = matches [0]
                else:
                    matches = matches[1]
                #print(matches, "1")
                claim_value = re.search("(\\d+\\W)+", matches).group()
                #print(claim_value)
            except Exception as e:
                claim_value="0"
                print("pdf_claim_value() error = ",e)
        # Appending claim_value to CVL list 
        if(claim_value=="0"):
            CVL.append("0")
            print("ClaimValue = ",claim_value)
        else:
            CVL.append(claim_value)
            print("ClaimValue = ",claim_value)
        #print("ITERATION  at pdf_claim_value() = ",CASE_NUM)
        os.remove('C:\\Users\\Dishant\\Downloads\\property_value.jpg')
        os.remove(filename)
    except Exception as e:
        print("Error at pdf_claim_value()",e)
        
        

"""FOR CLAIM VALUE CONVERTING TO INTEGER"""
def claim_val_converter(string_value):
    C_Value = ""
    try:
        for i in string_value.split(".")[0]: 
            if(i.isdigit()):

                C_Value = C_Value + i
        return int(C_Value)
    except Exception as e:
        C_Value = "NA"
        return C_Value

    

################################################################################################################################


from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pdf2image import convert_from_path
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import pyautogui
import requests
import random
import PyPDF2 
import json
import mouse
import time 
import glob
import os
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe"
base_dir = os.path.abspath(os.path.dirname("__file__"))

# Different Proxys : =    192.41.71.221:3128   50.246.120.125:8080     
PROXY = "192.41.71.199:3128"

options = webdriver.ChromeOptions()

appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}

options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False 
# options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
path = "C:\\Users\\Dishant\\Desktop\\chromedriver\\chromedriver"
global driver 
driver = webdriver.Chrome(executable_path = path , options = options)

#Step1
login() 

#Step2
collect_data_from_search() 
time.sleep(10)

#Step3
global CVL
CVL = []
try:
    
    for i in range(len(data_row)):
        driver.find_element_by_xpath("//*[@id='gridSearchResults']/tbody/tr["+str(i+1)+"]/td[3]/a").click()   
        #Web page load completely
        time.sleep(10)
        # PDF
        # Searching For PDF
        table = driver.find_element_by_id("gridDockets")
        rows = table.find_elements_by_tag_name("tr")

        # Gathering Data From GRID_DOCKETS {PDF search}
        gridDockets = []
        for i in range(1,len(rows)):
            gridDockets.append(rows[i].text)
        print("Lenght of Data Collected From Rows = ",len(gridDockets))
        # Searching for PDF present or not
        # imgCol
        for caught,data in enumerate(gridDockets):
            if ( "VALUE OF REAL PROPERTY OR MORTGAGE FORECLOSURE CLAIM:" in data ):
                print("PDF Found index = ",caught)
                break

        # Buttoned image
        x=0
        for i in range(caught):
            try:
                re.search("\d{1}\s\s\s\d{1,2}\/\d{1,2}\/\d{4}",gridDockets[i]).group()
                img = True
                x+=1
                #print("True",x)
            except Exception as e:
                pass
        #Clicking on the PDF imaged button
        javaScript = 'document.getElementsByClassName("icon-fixed-width icon-large icon-file-alt")['+str(x+1)+'].click();'
        driver.execute_script(javaScript)

        time.sleep(2)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(15)
        pyautogui.moveTo(1319, 38)
        time.sleep(5)
        pyautogui.moveTo(1212, 176)
        pyautogui.click()
        time.sleep(1)
        pyautogui.press('enter')
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        # To get back to search results
        driver.execute_script("window.history.go(-1)")
        #to load the page with all classes
        time.sleep(8)
        # Calling PDF_claim_value_function
        pdf_claim_value()
        time.sleep(3)
        print("Iteration = ",i)
except Exception as e:
    print("Error at Step 3 = ",e)
    CVL.append("NO PDF")
    print("Iteration = ",i)

driver.quit()
#Updating and Creating New Dataset
import pandas as pd
dataset = pd.read_csv("Martin_County_Try_Data.csv")
del dataset["Unnamed: 0"]
dataset["RealValueString"] = CVL
#RealValue
global RV
RV=[]
for i in CVL:
    RV.append(claim_val_converter(i))
dataset["RealValue"] = RV
os.remove('Martin_County_Try_Data.csv')
dataset.to_csv('Martin_County_Try_Data.csv')
dataset

## Step 3

In [ ]:

def login():
    """ LOGIN INTO THE WEBSITE WITH ALL DETAILS """
    try:
        driver.get("https://court.martinclerk.com")
        #To load Website Completely
        time.sleep(3)
        #To enter the dates from and to
        DateFrom = driver.find_element_by_id('openedFrom')
        DateFrom.click()
        DateFrom.send_keys("03162020")

        DateTo = driver.find_element_by_id('openedTo')
        DateTo.click()
        DateTo.send_keys("03222020")

        driver.find_element_by_xpath("//*[@id='courTypesButton']").click()
        driver.find_element_by_xpath("//*[@id='courTypesMultiSelect']/ul/li[1]/a/label/input").click()
        time.sleep(2) 
        driver.find_element_by_xpath("//*[@id='courTypesMultiSelect']/ul/li[3]/a/label/input").click()

        #To Get the next DropDownList
        time.sleep(10)
        driver.find_element_by_xpath("//*[@id='courTypesButton']").click()
        time.sleep(2)

        driver.find_element_by_xpath("//*[@id='caseTypesButton']").click()
        driver.find_element_by_xpath("//*[@id='caseTypesMultiSelect']/ul/li[1]/a/label/input").click()
        time.sleep(1)
        #To select the Categories
        for i in range(69,78):
            driver.find_element_by_xpath("//*[@id='caseTypesMultiSelect']/ul/li["+str(i)+"]/a/label/input").click()

        Captcha_Entry_Box = driver.find_element_by_xpath("//*[@id='mainTableContent']/tbody/tr/td/table/tbody/tr[2]/td[2]/div/div[3]/form/input[2]")
        Captcha_Entry_Box.click()

        time.sleep(3)

        captcha_additon = int(input("Enter the additon check the headless browser :\t"))
        Captcha_Entry_Box.send_keys(str(captcha_additon))

        #To click on Search Button
        Search_Button = driver.find_element_by_xpath("//*[@id='searchButton']")
        Search_Button.click()

        #To Load all the classes on webpage
        time.sleep(5)
    except Exception as e:
        print("Error at Login() = ",e)
        print("BURN THE CODE AGAIN....!!")

def collect_data_from_search():
    """ COLLECT DATA FROM SEARCH RESULT TO USE FOR ITERATIONS """
    try:
        table = driver.find_element_by_id("gridSearchResults")
        rows = table.find_elements_by_tag_name("tr")

        global data_row
        data_row = []
        for index in range(1,len(rows)):
            data_row.append(rows[index].text)
        print("Lenght of Data Collected From Rows = ",len(data_row))
    except Exception as e:
        print("Error at collect_data_from_search() = ",e)


"""FOR CLAIM VALUE CONVERTING TO INTEGER"""
def claim_val_converter(string_value):
    C_Value = ""
    try:
        for i in string_value.split(".")[0]: 
            if(i.isdigit()):

                C_Value = C_Value + i
        return int(C_Value)
    except Exception as e:
        C_Value = "NA"
        return C_Value



"""CODE FOR PDF address extraction"""
def PDFadd():
    try: 
        global add
        #Using GLOB module
        fn = glob.glob("C:\\Users\\Dishant\\Downloads\\*.pdf")
        FN = fn[0]
        print("Filename = ",FN)
        filename = FN       
        pages = convert_from_path(filename, 500)
        print("\nNumber of Pages found in PDF = ",len(pages))
        trials=0
        start = ""
        end = ""
        add=""
        found = False
        for page in pages:
            add=""
            page.save('C:\\Users\\Dishant\\Downloads\\add_img.jpg', 'JPEG')
            address_txt = str(((pytesseract.image_to_string(Image.open('C:\\Users\\Dishant\\Downloads\\add_img.jpg'))))).lower()
            try:
                add=""
                # If UNKNOWN is there 
                start = re.search(r'unknown',address_txt).span()
                end = address_txt[start[1]:]
                Unkonwn_Found = True
                if(Unkonwn_Found):
                    try:
                        add=""
                        print("UNKNOWN FOUND")
                        ############ First Attempt
                        try:
                            add = re.search("\d{2,5}\s[\b\w,@\s]*\sfl\s\d{4,5}\-\d{3,5}", end).group()
                            print(add)
                            print("Address_Found in Attempt = 1")
                            print("1")
                            comp_address.append(add)
                            break
                        except Exception as e:
                            add = re.search(r'\d{2,5}\s[\b\w,@\s]*\sfl\s\d{4,5}', end).group()
                            print(add)
                            print("Address_Found in (Exception)Attempt = 1")
                            print("2")
                            comp_address.append(add)
                            break
                    except Exception as e:
                        add=""
                        ############ Second Attempt
                        try:
                            add = re.search("\d{2,5}\s[\b\w,@\s]*\sflorida\s\d{4,5}\-\d{3,5}", end).group()
                            print(add)
                            print("Address_Found in Attempt = 2")
                            print("3")
                            comp_address.append(add)
                            break
                        except Exception as e:
                            add = re.search("\d{2,5}\s[\b\w,@\s]*\sflorida\s\d{4,5}", end).group()
                            print(add)
                            print("Address_Found in (Exception)Attempt = 2")
                            print("4")
                            comp_address.append(add)
                            break

            except Exception as e:
                add=""
                try:
                    add=""
                    print("UNKNOWN NOT FOUND")
                    ############ First Attempt
                    try:
                        add = re.search("\d{2,5}\s[\b\w,@\s]*\sfl\s\d{4,5}\-\d{3,5}", address_txt).group()
                        print(add)
                        print("Address_Found in Attempt = 1")
                        print("5")
                        comp_address.append(add)
                        break
                    except Exception as e:
                        add = re.search(r'\d{2,5}\s[\b\w,@\s]*\sfl\s\d{4,5}', address_txt).group()
                        print(add)
                        print("Address_Found in (Exception)Attempt = 1")
                        print("6")
                        comp_address.append(add)
                        break
                except Exception as e:
                    add=""
                    ############ Second Attempt
                    try:
                        add = re.search("\d{2,5}\s[\b\w,@\s]*\sflorida\s\d{4,5}\-\d{3,5}", address_txt).group()
                        print(add)
                        print("Address_Found in Attempt = 2")
                        print("7")
                        comp_address.append(add)
                        break
                    except Exception as e:
                        add = re.search("\d{2,5}\s[\b\w,@\s]*\sflorida\s\d{4,5}", address_txt).group()
                        print(add)
                        print("Address_Found in (Exception)Attempt = 2")
                        print("8")
                        comp_address.append(add)
                        break
    except Exception as e:
        print("ERROR")
        # If add is empty .....  
    if(add==""):
        comp_address.append("NAF")
    os.remove('C:\\Users\\Dishant\\Downloads\\add_img.jpg')
    os.remove(filename)
    #print("ITERATION at PDFadd() exception = ",CASE_NUM)
    #os.close(filename)

    

################################################################################################################################


from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pdf2image import convert_from_path
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import pyautogui
import requests
import random
import PyPDF2 
import json
import mouse
import time 
import glob
import os
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe"
base_dir = os.path.abspath(os.path.dirname("__file__"))

# Different Proxys : =    192.41.71.221:3128   50.246.120.125:8080     
PROXY = "192.41.71.199:3128"

options = webdriver.ChromeOptions()

appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}

options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False 
# options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
path = "C:\\Users\\Dishant\\Desktop\\chromedriver\\chromedriver"
global driver 
driver = webdriver.Chrome(executable_path = path , options = options)

#Step1
login() 

#Step2
collect_data_from_search() 
time.sleep(10)


# taking values from dataset
import pandas as pd
dataset = pd.read_csv("Martin_County_Try_Data.csv")


#Step3
global comp_address
comp_address = []
try:
    
    for xyz in range(len(data_row)):
        driver.find_element_by_xpath("//*[@id='gridSearchResults']/tbody/tr["+str(xyz+1)+"]/td[3]/a").click()   
        #Web page load completely
        time.sleep(10)
        # PDF
        # Searching For PDF
        table = driver.find_element_by_id("gridDockets")
        rows = table.find_elements_by_tag_name("tr")

        # Gathering Data From GRID_DOCKETS {PDF search}
        gridDockets = []
        for i in range(1,len(rows)):
            gridDockets.append(rows[i].text)
        print("Lenght of Data Collected From Rows = ",len(gridDockets))

        # Searching for PDF present or not
        # imgCol
        for caught,data in enumerate(gridDockets):
            if ( "SUMMONS ISSUED TO" in data ):
                print("PDF Found index = ",caught)
                break

        # Buttoned image
        x=0
        for i in range(caught):
            try:
                re.search("\d{1}\s\s\s\d{1,2}\/\d{1,2}\/\d{4}",gridDockets[i]).group()
                img = True
                x+=1
                #print("True",x)
            except Exception as e:
                pass

        #Clicking on the PDF imaged button
        javaScript = 'document.getElementsByClassName("icon-fixed-width icon-large icon-file-alt")['+str(x+1)+'].click();'
        driver.execute_script(javaScript)

        time.sleep(2)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(60)
        pyautogui.moveTo(1319, 38)
        time.sleep(5)
        pyautogui.moveTo(1212, 176)
        pyautogui.click()
        time.sleep(1)
        pyautogui.press('enter')
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(2)
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        # To get back to search results
        driver.execute_script("window.history.go(-1)")
        #to load the page with all classes
        time.sleep(8)

        # Collecting Address
        PDFadd()
        
        time.sleep(3)
        print("Iteration = ",xyz)
except Exception as e:
    print("Error at Step 3 = ",e)
    comp_address.append("NAF")
    print("Iteration = ",xyz)

#Creating new DataSet

import pandas as pd
dataset = pd.read_csv("Martin_County_Try_Data.csv")
del dataset["Unnamed: 0"]
dataset["Comp_Address"] = comp_address
os.remove('Martin_County_Try_Data.csv')
dataset.to_csv('Martin_County_Try_Data.csv')
dataset


## Step 4

In [ ]:

"""FOR GOING TO WESITE FOR PROPERTY VALUE"""
def prop_val(x,y,z):
    try:
        driver.get("http://www.totalviewrealestate.com/")  
        time.sleep(10)   
        driver.find_element_by_xpath('//*[@id="header"]/form/p/input[1]').send_keys(x) 
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="header"]/form/p/input[2]').send_keys(y) 
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="header"]/form/p/input[3]').send_keys(z) 
        time.sleep(4)
        driver.find_element_by_xpath('//*[@id="header"]/form/input').click() #SUBMIT
        time.sleep(17)
        data = driver.find_element_by_xpath('//*[@id="leftpi"]/p[2]').text
        print(data)
        lis.append(data)
        time.sleep(5)
    except Exception as e:
        print("Error at pop_val() = ",e)
        lis.append("NAF")


"""FOR CLAIM VALUE CONVERTING TO INTEGER"""
def claim_val_converter(string_value):
    C_Value = ""
    try:
        for i in string_value.split(".")[0]: 
            if(i.isdigit()):

                C_Value = C_Value + i
        return int(C_Value)
    except Exception as e:
        C_Value = "NA"
        return C_Value


"""FOR HIGH VALUATION VALUE"""
def HVVF():
    high_value = ""
    for i in HVS: 
        if(i.isdigit()):
            high_value = high_value + i
    return high_value

"""FOR LOW VALUATION VALUE"""
def LVVF():
    low_value = ""
    for i in LVS: 
        if(i.isdigit()):
            low_value = low_value + i
    return low_value
         
    
#######################################################


from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pdf2image import convert_from_path
# import win32com.client as wincl       # Speech Module
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import pyautogui
import requests
import random
import PyPDF2 
import json
import mouse
import time 
import os
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe"
base_dir = os.path.abspath(os.path.dirname("__file__"))

# Different Proxys : =    192.41.71.199:3128   50.246.120.125:8080      
PROXY =   "138.197.32.120:3128"  #"157.230.44.213:8080"  #"138.197.32.120:3128"  #"74.121.98.90:8080"  
            #"192.144.215.244:3128"  #"157.230.44.213:8080"   #"74.121.98.90:8080"    #"192.41.71.221:3128"
options = webdriver.ChromeOptions()
appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}
options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False #True  
options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
path = "C:\\Users\\Dishant\\Desktop\\chromedriver\\chromedriver"

global driver , lis
lis = []

driver = webdriver.Chrome(executable_path = path , options = options)

"""MAIN PROGRAM"""
#Reading Address from Dataset
dataset = pd.read_csv("Martin_County_Try_Data.csv")
dataset.head(6)

new = []
for i in dataset["Comp_Address"]: new.append(i)
print(len(new))
########### Sending data to get ratios and values for calculation
for i in range(len(new)):
#     print(new[i])
    if(new[i]=="NAF"):
        lis.append("NAF")
        print("NAF","iteration  = ",i)
        continue
    try:
        try:
            
            demo = new[i].split("\n")
            Address = demo[0]
            City = demo[3].split(",")[0]
            pin =  demo[3].split(",")[1].split(" ")[2]
            prop_val(Address,City,pin)
            print(Address,"|",City,"|",pin,"iteration  = ",i)
            
        except Exception as e:
            
            demo = new[i].split("\n")
            Address = demo[0]
            City = demo[1].split(",")[0]
            pin =  demo[1].split(",")[1].split(" ")[2]
            prop_val(Address,City,pin)
            print(Address,"|",City,"|",pin,"iteration  = ",i)
            
    except Exception as e:
        print("Error = ",e,"  iteration  = ",i)
        lis.append("NAF")
        continue            
driver.quit()


#Cleaning the gathered data into four lists for Calculations      
list_HVS = []
list_HVV = []
list_LVS = []
list_LVV = []

""" FOR EXTRACTING HIGH LOW VALUE OF PROPRERTY """
for i in range(len(lis)):
    demo = lis[i]
    try:
        #print(demo)
        start = re.search("Value\sRange",demo).span()
        #print(start)
        end = re.search("Change\:\s",demo).span()
        #print(end)
        got = demo[start[1]:end[0]]
        got.split("to")
        HVS = got.split("to")[1].split("\n")[0]
        HVV = HVVF()
        LVS = got.split("to")[0].split(" ")[1]
        LVV = LVVF()
        print("\n")
        print("Low Value String = ",LVS)
        print("Low Value = ",LVV)
        print("High Value String = ",HVS)
        print("High Value = ",HVV)
        list_HVS.append(HVS)
        list_HVV.append(int(HVV))
        list_LVS.append(LVS)
        list_LVV.append(int(LVV))
    except Exception as e:
        HVS = "NA"
        HVV = "NA"
        LVS = "NA"
        LVV = "NA"
        print("\n")
        print("Low Value String = ",LVS)
        print("Low Value = ",LVV)
        print("High Value String = ",HVS)
        print("High Value = ",HVV)
        list_HVS.append(HVS)
        list_HVV.append(HVV)
        list_LVS.append(LVS)
        list_LVV.append(LVV)
        
# print(len(list_HVS))
# print(len(list_LVS))
# print(len(list_HVV))
# print(len(list_LVV))

# """FOR CALCULATIONS"""
RV = []
for i in dataset["RealValue"]: RV.append(i)
print(len(RV))

Avg_App_V_L=[]
Avg_Est_V_L = []
Acq_Ratio_L = []


for i in range(len(RV)):
    try:
        real_value = RV[i]
        high = list_HVV[i]
        #print(high)
        low = list_LVV[i]
        #print(low)

        if(high=="NA" or low == "NA"):
            Avg_App_V = "NA"   
            Avg_App_V_L.append(Avg_App_V) #Average Approximate Value
            Avg_Est_V = "NA" 
            Avg_Est_V_L.append(Avg_Est_V) #Average Estimated Value
            Acq_Ratio = "NA"
            Acq_Ratio_L.append(Acq_Ratio) #Acquisation Ratio
        else:
            Avg_App_V = (high+low)/2   
            Avg_App_V_L.append(Avg_App_V) #Average Approximate Value
            if(real_value == "NA"):
                Avg_Est_V = "NA"
                Avg_Est_V_L.append(Avg_Est_V) #Average Estimated Value
                Acq_Ratio = "NA"
                Acq_Ratio_L.append(Acq_Ratio) #Acquisation Ratio

            else:
                Avg_Est_V = Avg_App_V - real_value 
                Avg_Est_V_L.append(Avg_Est_V) #Average Estimated Value
                Acq_Ratio = (real_value / Avg_App_V) * 100
                Acq_Ratio_L.append(Acq_Ratio) #Acquisation Ratio
    except Exception as e:
        Avg_App_V = "NA"   
        Avg_App_V_L.append(Avg_App_V) #Average Approximate Value
        Avg_Est_V = "NA" 
        Avg_Est_V_L.append(Avg_Est_V) #Average Estimated Value
        Acq_Ratio = "NA"
        Acq_Ratio_L.append(Acq_Ratio) #Acquisation Ratio

        
""" Getting Address , City , Pincode """
new = []
for i in dataset["Comp_Address"]:
    new.append(i)
print(len(new))
# new 

"""ADDING AQUIRED DATA TO DATAFRAME"""
address = []
city = []
zipcode  = []

for i in range(len(new)):
    if(new[i]=="0"):
        address.append("0")
        city.append("0")
        zipcode.append("0")
        continue
    try:
        try:

            demo = new[i].split("\n")
            Address = demo[0]
            City = demo[3].split(",")[0]
            pin =  demo[3].split(",")[1].split(" ")[2]
            address.append(Address)
            city.append(City)
            zipcode.append("FL " + pin)
            print(Address,"|",City,"|",pin,"iteration  = ",i)            

        except Exception as e:

            demo = new[i].split("\n")
            Address = demo[0]
            City = demo[1].split(",")[0]
            pin =  demo[1].split(",")[1].split(" ")[2]
            address.append(Address)
            city.append(City)
            zipcode.append("FL " + pin)
            print(Address,"|",City,"|",pin,"iteration  = ",i)   
            
    except Exception as e:
        print("Error = ",e,"  iteration  = ",i)
        address.append("0")
        city.append("0")
        zipcode.append("0")
        continue

# print(len(Avg_App_V_L),len(Avg_Est_V_L),len(Acq_Ratio_L))
# print(len(address),len(city),len(zipcode))

#Making new dataset
del dataset["Unnamed: 0"]
dataset['Address'] = address
dataset['City'] = city
dataset['Zipcode'] = zipcode
dataset['HighValuationString'] = list_HVS
dataset['HighValuationValue'] = list_HVV
dataset['LowValuationString'] = list_LVS
dataset['LowValuationValue'] = list_LVV
dataset['AverageApproximateValue'] = Avg_App_V_L
dataset['AverageEstimateValue'] = Avg_Est_V_L
dataset['AcquisitionRatio'] = Acq_Ratio_L
del dataset["Comp_Address"]
county_name = []
for i in range(len(lis)): county_name.append("Martin County")
dataset['CountyName'] = county_name

os.remove("Martin_County_Try_Data.csv")
dataset.to_csv('Martin_County_Try_Data.csv')   # Final CSV File...
dataset



In [ ]:
dataset

In [ ]:
# Point(x=1212, y=176) #download
# Point(x=1319, y=38) #close
# time.sleep(10)
# pyautogui.hotkey("ctrlleft", "p") 